- authors: Eric Kernfeld, Macrina Lobo

In [ ]:
# Set up namespace
setwd("~/Dropbox/FINAL_CO/")
library("ggplot2")
library("magrittr")
library("Seurat")
library("dplyr")
results_path = "erics_code_network_analysis"
dir.create(results_path)
# Load relevant RNA data
dge = readRDS("~/Dropbox/pharynx_celloracle/objects/roadmap/RoadmapSeurat3.rds") %>% UpdateSeuratObject()
dge_pouch = readRDS("~/Dropbox/pharynx_celloracle/objects/roadmap/pouch_object.Rdata") %>% UpdateSeuratObject()
pg = VariableFeatures(dge)
fig1_colors = read.csv("~/Dropbox/pharynx_celloracle/objects/roadmap/roadmap_colors.csv", stringsAsFactors = F)
fig1_colors = setNames(fig1_colors$colour, fig1_colors$cell_type)
expression_by_cluster = Seurat::AverageExpression((dge))
# Load filtered networks from CO
filtered_network_files = "filtered_GRN_10k_0701_2021" %>% list.files(full = T)
names(filtered_network_files) = basename(filtered_network_files) %>% gsub(".csv$", "", .)
cell_type_networks = lapply(filtered_network_files, read.csv, header = T, stringsAsFactors = F, row.names = 1)
cell_type_networks = cell_type_networks[order(as.numeric(names(cell_type_networks)))]
for(cc in names(cell_type_networks)){
  cell_type_networks[[cc]][["cluster"]] = cc
}
# short detour: log the Foxn1 targets
foxn1_targets = cell_type_networks %>% lapply(subset, source=="Foxn1", select = c("target"), drop = T) %>% Reduce(f = union)
write.csv(foxn1_targets, "foxn1_direct_targets.csv")
# Properties of each network: degree distribution and edge weight distribution
# degree (outdegree by default; for in, set source_or_target = "target")
get_degree = function(X, eligible_genes = pg, source_or_target = "source"){
  if(length(eligible_genes)>0){
    X %<>% subset(target %in% pg) 
  } 
  X[c("target", "cluster")] %>% table %>% as.data.frame() %>% set_colnames(c("gene", "cluster", "degree"))
}
outdegree = cell_type_networks %>% lapply(get_degree, eligible_genes = NULL, source_or_target = "target") 
outdegree %<>% Reduce(f = rbind)
dim(outdegree)
head(outdegree)
outdegree_order = outdegree %>% 
  group_by(cluster) %>%
  summarize(median_degree = median(degree)) %>%
  arrange(median_degree)
outdegree$cluster %<>% factor(outdegree_order$cluster)
ggplot(outdegree) + 
  geom_violin(aes(y = degree, x = cluster)) 



In [ ]:
ggsave("erics_code_network_analysis/indegree.pdf", width = 6, height = 2)
# edge weight
edge_weight_order = cell_type_networks %>% 
  lapply(extract, c("coef_mean", "cluster")) %>%
  Reduce(f = rbind) %>% 
  group_by(cluster) %>%
  summarize(median_absolute_edge_weight = median(abs(coef_mean))) %>%
  arrange(median_absolute_edge_weight)
cell_type_networks %>% 
  lapply(extract, c("coef_mean", "cluster")) %>% 
  Reduce(f = rbind) %>%
  mutate(cluster = cluster %>% factor(levels = edge_weight_order$cluster)) %>%
  ggplot() + 
  geom_violin(aes(y = coef_mean, x = cluster)) + 
  scale_y_continuous(
    trans = scales::trans_new(
      name = "signed_log",
      transform = function(x) sign(x)*log2(abs(x)),
      inverse   = function(x) sign(x)*2^(abs(x))
    ), 
    breaks = c(1, 2, 5, 10, 20, 50, 100, 200) %o% c(-1, 1) %>% c %>% sort
  )
ggsave("erics_code_network_analysis/edge_weight.pdf", width = 6, height = 2)


# How much do the networks have in common?
get_common_links = function(net1, net2){
  net1 %<>% mutate(std = paste(source, target, sign(coef_mean), sep = "__"))
  net2 %<>% mutate(std = paste(source, target, sign(coef_mean), sep = "__"))
  length(intersect(net1$std, net2$std))
}
overlaps = matrix(0, nrow = 28, ncol = 28)
for(i in seq_along(cell_type_networks)){
  for(j in seq_along(cell_type_networks)){
    if(j>=i){next}
    overlaps[i, j] = get_common_links(cell_type_networks[[i]], cell_type_networks[[j]])
  }
}
overlaps = overlaps + t(overlaps)
cluster_order = seriation::seriate(as.dist(exp(-overlaps/3e4))) %>% (seriation::get_order) %>% extract(names(cell_type_networks), .) 
overlaps_long = overlaps %>%
  reshape2::melt() %>% 
  mutate(net1 = names(cell_type_networks)[Var1] %>% factor(levels = cluster_order), 
         net2 = names(cell_type_networks)[Var2] %>% factor(levels = cluster_order),
         overlap = value) 
ggplot(overlaps_long) + 
  geom_tile(aes(x = net1, y = net2, fill = overlap/3e4) ) + 
  scale_fill_viridis_c(option = "viridis", name = "Proportion matching\n(source, target, and\ndirection)") + 
  cowplot::theme_cowplot() + 
  theme(axis.text.x = element_text(angle  = 90, hjust = 1, vjust = 0.5)) + 
  ggtitle("Overlap between cell type-specific networks") + 
  coord_fixed() + 
  ggsave("erics_code_network_analysis/subnet_overlap.pdf", width = 6, height = 6)







In [ ]:
# Choose top regulators by cluster using the CellOracle scores
scores = read.csv("scores_unnormalized.csv", stringsAsFactors = F)
colnames(scores)[[1]] = "gene"
# quick detour: thymus vs parathyroid
scores_wide = scores %>%
  subset(cluster %in% c(4, 26)) %>% 
  tidyr::pivot_wider(id_cols = "gene", names_from = "cluster", values_from = "degree_centrality_out") 
ggplot(scores_wide, mapping = aes(x = `4`, y = `26`, label = gene))  +
  geom_point() + 
  ggrepel::geom_text_repel(data = scores_wide %>% subset(abs(`4`-`26`)>0.05)) + 
  ggtitle("Thymus vs parathyroid") + 
  ggsave("erics_code_network_analysis/Thymus vs parathyroid.pdf", width = 5, height = 5)
# proceed




- fix Nkx2-1

In [17]:
library(stringr)

In [20]:
new_var = c()
for (entry in rownames(expression_by_cluster$RNA)){
    new_var = c(new_var,str_replace(entry,'\\.','-'))}

In [21]:
new_var

[1] "Xkr4"             "Gm1992"           "Gm37381"         
    [4] "Rp1"              "Rp1-1"            "Sox17"           
    [7] "Gm37323"          "Mrpl15"           "Lypla1"          
   [10] "Gm37988"          "Tcea1"            "Rgs20"           
   [13] "Gm16041"          "Atp6v1h"          "Oprk1"           
   [16] "Npbwr1"           "Rb1cc1"           "X4732440D04Rik"  
   [19] "Fam150a"          "St18"             "Pcmtd1"          
   [22] "Gm26901"          "Gm30414"          "Sntg1"           
   [25] "Rrs1"             "Adhfe1"           "X3110035E14Rik"  
   [28] "Gm29520"          "Mybl1"            "Vcpip1"          
   [31] "X1700034P13Rik"   "Sgk3"             "Mcmdc2"          
   [34] "Snhg6"            "Tcf24"            "Ppp1r42"         
   [37] "Gm15818"          "Cops5"            "Cspp1"           
   [40] "Arfgef1"          "Cpa6"             "Prex2"           
   [43] "A830018L16Rik"    "Gm17644"          "Gm29663"         
   [46] "Sulf1"            "Slco5a1"          "Gm29283"         
   [49] "Prdm14"           "Ncoa2"            "Gm29570"         
   [52] "Tram1"            "Lactb2"           "Xkr9"            
   [55] "Eya1"             "Gm9947"           "Msc"             
   [58] "Trpa1"            "Kcnb2"            "Terf1"           
   [61] "Sbspon"           "X4930444P10Rik"   "Rpl7"            
   [64] "Rdh10"            "Gm28095"          "Stau2"           
   [67] "Gm7568"           "Ube2w"            "Tceb1"           
   [70] "D030040B21Rik"    "Tmem70"           "Ly96"            
   [73] "Gm28376"          "Jph1"             "Gm28783"         
   [76] "Gdap1"            "Gm28784"          "Pi15"            
   [79] "Gm28154"          "Gm16070"          "Crispld1"        
   [82] "Gm28153"          "Crisp4"           "Defb18"          
   [85] "Defb41"           "Gm29006"          "Gm15825"         
   [88] "Tfap2d"           "Tfap2b"           "Gm28340"         
   [91] "Pkhd1"            "X4930486I03Rik"   "Gm28653"         
   [94] "Il17a"            "Il17f"            "Mcm3"            
   [97] "Gm28065"          "X6720483E21Rik"   "Paqr8"           
  [100] "Efhc1"            "Tram2"            "Gm28287"         
  [103] "Tmem14a"          "Gsta3"            "Gm28836"         
  [106] "Khdc1a"           "Khdc1c"           "Khdc1b"          
  [109] "Kcnq5"            "Gm26580"          "Rims1"           
  [112] "X4933415F23Rik"   "Gm27028"          "Gm29107"         
  [115] "Ogfrl1"           "Gm28822"          "B3gat2"          
  [118] "Smap1"            "Sdhaf4"           "Fam135a"         
  [121] "Col9a1"           "Col19a1"          "Lmbrd1"          
  [124] "Gm28237"          "Gm29414"          "Adgrb3"          
  [127] "X4931408C20Rik"   "Gm5524"           "Gm597"           
  [130] "Gm9898"           "Phf3"             "Ptp4a1"          
  [133] "Gm29669"          "Gm28644"          "X4931428L18Rik"  
  [136] "Gm29128"          "Lgsn"             "Pih1d3"          
  [139] "Khdrbs2"          "Gm5415"           "Gm37724"         
  [142] "Gm37591"          "Prim2"            "Rab23"           
  [145] "Bag2"             "Zfp451"           "Bend6"           
  [148] "Dst"              "Gm37958"          "Gm37233"         
  [151] "Ccdc115"          "Gm28306"          "Imp4"            
  [154] "Ptpn18"           "X4930568A12Rik"   "Prss39"          
  [157] "Cfc1"             "Prss40"           "X1700101I19Rik"  
  [160] "Amer3"            "Arhgef4"          "Gm38336"         
  [163] "Fam168b"          "Plekhb2"          "Gm37068"         
  [166] "Gm28415"          "X1110002O04Rik"   "Gm37146"         
  [169] "Gm33222"          "X4930535G08Rik"   "Hs6st1"          
  [172] "Gm37335"          "Gm33280"          "Uggt1"           
  [175] "Neurl3"           "Arid5a"           "X4930403P22Rik"  
  [178] "Kansl3"           "Fer1l5"           "Lman2l"          
  [181] "Cnnm4"            "Cnnm3"            "Gm38033"         
  [184] "Ankrd23"          "Gm43543"   

In [22]:
rownames(expression_by_cluster$RNA) = new_var

In [30]:
for (entry in as.vector(scores['gene'])){
    if ('Nkx2.1' == entry){
        print(entry)
    }
}

Warning message in if ("Nkx2.1" == entry) {:
“the condition has length > 1 and only the first element will be used”


In [34]:
scores[scores['gene']=='Nkx2-1',]

gene,degree_all,degree_in,degree_out,clustering_coefficient,clustering_coefficient_weighted,degree_centrality_all,degree_centrality_in,degree_centrality_out,betweenness_centrality,⋯,assortative_coefficient,average_path_length,community_random_walk,module,connectivity,participation,role,cluster,expression_specificity,betweenness_centrality_normalized
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,⋯,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>,<int>,<dbl>,<dbl>
Nkx2-1,92,4,88,0.01505017,0.01647838,0.025662483,0.0011157601,0.024546722,1654,⋯,-0.2787503,3.504223,13,1,4.59727686,0.3353025,Connector Hub,12,85.75037,2.2296113
Nkx2-1,10,10,0,0.28888889,0.31915010,0.001853568,0.0018535681,0.000000000,0,⋯,-0.2556780,2.714330,1,3,0.03546159,0.6800000,Connector,15,20.80914,0.0000000
Nkx2-1,64,7,57,0.04365079,0.05088298,0.013375131,0.0014629049,0.011912226,988,⋯,-0.0943364,2.859716,9,1,1.91943325,0.6020508,Peripheral,17,107.32699,1.3318355
Nkx2-1,64,13,51,0.06746032,0.11837001,0.010654237,0.0021641418,0.008490095,1448,⋯,-0.2015887,2.724020,8,2,1.09892285,0.6215820,Connector,22,579.43271,1.9519209
Nkx2-1,7,7,0,0.14285714,0.13070198,0.001561106,0.0015611062,0.000000000,0,⋯,-0.1907596,2.649655,2,0,0.06886374,0.5714286,Peripheral,24,31.18209,0.0000000
Nkx2-1,64,4,60,0.02678571,0.05293994,0.015044664,0.0009402915,0.014104372,361,⋯,-0.1746149,2.280680,1,1,1.73120129,0.4785156,Peripheral,27,518.07309,0.4866322


In [45]:
remove(cluster)

Warning message in remove(cluster):
“object 'cluster' not found”


In [49]:
scores = read.csv("scores_unnormalized.csv", stringsAsFactors = F)
colnames(scores)[[1]] = "gene"

get_specificity = function(gene, cluster){
  expression_by_cluster$RNA[gene, as.character(cluster)] / median(unlist(expression_by_cluster$RNA[gene,]))
}


# clusters_use = c(9,2,22,12,8) 
#clusters_use = c(22) 

scores %<>% dplyr::mutate(expression_specificity = mapply(get_specificity, gene, cluster))
scores$expression_specificity[is.nan(scores$expression_specificity)] = 0
scores$expression_specificity %<>% pmin(1000)
scores %<>%
  dplyr::group_by(gene) %>%
  dplyr::mutate(betweenness_centrality_normalized = betweenness_centrality/mean(betweenness_centrality)) %>%
  #dplyr::mutate(betweenness_centrality_normalized = betweenness_centrality) %>%
  dplyr::ungroup()




for (clusters_use in c(26,4,12,22)){
#get_specificity("Foxn1", 4)

top_regulators = scores %>% 
  subset(cluster %in% clusters_use) %>% 
  subset(expression_specificity>=5) %>% 
  dplyr::group_by(cluster) %>%
  dplyr::top_n(betweenness_centrality_normalized, n=20) 
p = scores %>%
  subset(cluster %in% clusters_use) %>% 
  ggplot(aes(x = expression_specificity, 
             y = betweenness_centrality_normalized,
             colour = as.character(cluster),
             fill = as.character(cluster), 
             label = gene)) + 
  scale_y_continuous(trans = "log1p") + 
  scale_x_continuous(trans = "log1p") + 
  ggtitle("Top regulators by cluster") + 
  geom_point() + 
  # ggrepel::geom_text_repel(data = top_regulators,color='black',max.overlaps = Inf) + 
  scale_fill_manual(values = fig1_colors) +
  scale_colour_manual(values = fig1_colors) +
  facet_grid(~cluster) + 
  ggsave(file.path(results_path, paste(clusters_use,"Norm_bet_cen_vs_rna_expr.png",sep='_')), width = 6, height = 4)  

}


Warning message:
“Removed 4364 rows containing missing values (geom_point).”
Warning message:
“Removed 4899 rows containing missing values (geom_point).”
Warning message:
“Removed 3453 rows containing missing values (geom_point).”
Warning message:
“Removed 5858 rows containing missing values (geom_point).”


In [41]:

scores = read.csv("scores_unnormalized.csv", stringsAsFactors = F)
colnames(scores)[[1]] = "gene"

get_specificity = function(gene, cluster){
  expression_by_cluster$RNA[gene, as.character(cluster)] / median(unlist(expression_by_cluster$RNA[gene,]))
}


# clusters_use = c(9,2,22,12,8) 
for (clust in c(9,2,22,12,8) ){
print(clust)
# get_specificity("Foxn1", 4)
scores %<>% dplyr::mutate(expression_specificity = mapply(get_specificity, gene, clust))
scores$expression_specificity[is.nan(scores$expression_specificity)] = 0
scores$expression_specificity %<>% pmin(1000)
scores %<>%
  dplyr::group_by(gene) %>%
  dplyr::mutate(betweenness_centrality_normalized = betweenness_centrality/mean(betweenness_centrality)) %>%
  #dplyr::mutate(betweenness_centrality_normalized = betweenness_centrality) %>%
  dplyr::ungroup()

top_regulators = scores %>% 
  subset(cluster %in% clust) %>% 
  subset(expression_specificity>=5) %>% 
  dplyr::group_by(cluster) %>%
  dplyr::top_n(betweenness_centrality_normalized, n=20) 
p = scores %>%
  subset(cluster %in% clust) %>% 
  ggplot(aes(x = expression_specificity, 
             y = betweenness_centrality_normalized,
             colour = as.character(cluster),
             fill = as.character(cluster), 
             label = gene)) + 
  scale_y_continuous(trans = "log1p") + 
  scale_x_continuous(trans = "log1p") + 
  ggtitle("Top regulators by cluster") + 
  geom_point() + 
  ggrepel::geom_text_repel(data = top_regulators,color='black',max.overlaps = Inf) + 
  scale_fill_manual(values = fig1_colors) +
  scale_colour_manual(values = fig1_colors) +
  facet_grid(~cluster) + 
  ggsave(file.path(results_path, paste(cluster,"Norm_bet_cen_vs_rna_expr.pdf",sep='_')), width = 6, height = 4)  

    }

